In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv('train.csv')

In [11]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

age_median = df['Age'].median()
df['Age'] = df['Age'].fillna(age_median)

from scipy.stats import mode

mode_embarked = mode(df['Embarked'].dropna())[0][0]
df['Embarked'] = df['Embarked'].fillna(mode_embarked)

df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)

pd.get_dummies(df['Embarked'], prefix='Embarked').head(10)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)

df = df.drop(['Sex', 'Embarked'], axis=1)

cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]

df = df[cols]

train_data = df.values

C:\Users\darthjarjar\Anaconda3\lib\site-packages\scipy\stats\stats.py:250: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

parameter_grid = {
    'max_features': [0.5, 1.],
    'max_depth': [5., None]
}

grid_search = GridSearchCV(RandomForestClassifier(n_estimators = 100), parameter_grid, cv=5, verbose=3)


C:\Users\darthjarjar\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\darthjarjar\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [13]:
grid_search.fit(train_data[0:,2:], train_data[0:,0])

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] max_depth=5.0, max_features=0.5 .................................
[CV] ........ max_depth=5.0, max_features=0.5, score=0.804469 -   0.0s
[CV] max_depth=5.0, max_features=0.5 .................................

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s



[CV] ........ max_depth=5.0, max_features=0.5, score=0.826816 -   0.1s
[CV] max_depth=5.0, max_features=0.5 .................................
[CV] ........ max_depth=5.0, max_features=0.5, score=0.837079 -   0.0s
[CV] max_depth=5.0, max_features=0.5 .................................
[CV] ........ max_depth=5.0, max_features=0.5, score=0.780899 -   0.0s
[CV] max_depth=5.0, max_features=0.5 .................................
[CV] ........ max_depth=5.0, max_features=0.5, score=0.853107 -   0.0s
[CV] max_depth=5.0, max_features=1.0 .................................
[CV] ........ max_depth=5.0, max_features=1.0, score=0.810056 -   0.1s
[CV] max_depth=5.0, max_features=1.0 .................................
[CV] ........ max_depth=5.0, max_features=1.0, score=0.826816 -   0.0s
[CV] max_depth=5.0, max_features=1.0 .................................
[CV] ........ max_depth=5.0, max_features=1.0, score=0.842697 -   0.1s
[CV] max_depth=5.0, max_features=1.0 .................................
[CV] 

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    3.3s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [0.5, 1.0], 'max_depth': [5.0, None]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=3)

In [14]:
grid_search.grid_scores_

[mean: 0.82043, std: 0.02531, params: {'max_depth': 5.0, 'max_features': 0.5},
 mean: 0.82716, std: 0.02190, params: {'max_depth': 5.0, 'max_features': 1.0},
 mean: 0.81481, std: 0.03153, params: {'max_depth': None, 'max_features': 0.5},
 mean: 0.80920, std: 0.02547, params: {'max_depth': None, 'max_features': 1.0}]

In [15]:
sorted(grid_search.grid_scores_, key = lambda x: x.mean_validation_score)
grid_search.best_score_
grid_search.best_params_

{'max_depth': 5.0, 'max_features': 1.0}

In [16]:
model = RandomForestClassifier(n_estimators=100, max_features=0.5, max_depth=5.0)
model = model.fit(train_data[0:,2:],train_data[0:,0])

In [17]:
df_test = pd.read_csv('test.csv')

df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_median)

fare_means = df.pivot_table('Fare', index='Pclass', aggfunc='mean')
df_test['Fare'] = df_test[['Fare', 'Pclass']].apply(lambda x:
                            fare_means[x['Pclass']] if pd.isnull(x['Fare'])
                            else x['Fare'], axis=1)

df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')],
                axis=1)

df_test = df_test.drop(['Sex', 'Embarked'], axis=1)

test_data = df_test.values

output = model.predict(test_data[:,1:])

In [18]:
result = np.c_[test_data[:,0].astype(int), output.astype(int)]


df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])
df_result.to_csv('submissions/submissionTuned.csv', index=False)